In [ ]:
import os
from dataset import Dataset

__ORIG_WD__ = os.getcwd()
os.chdir(f"{__ORIG_WD__}/../data_collectors")

from ncbi_genome import NCBIGenome

os.chdir(__ORIG_WD__)


In [ ]:
dc = NCBIGenome()

In [ ]:
taxa = dc.getLocalTaxa()
accession_taxa_list = []
for taxon in taxa:
    for accession in dc.getLocalAccessions(taxon):
        accession_taxa_list.append((accession, taxon))

In [ ]:
dataset = Dataset(accession_taxa_list)

In [ ]:
import os
from dataset import Dataset
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Uncomment to disable GPU
import numpy as np

__ORIG_WD__ = os.getcwd()

os.chdir(f"{__ORIG_WD__}/../data_collectors/")
from covid19_genome import Covid19Genome


os.chdir(__ORIG_WD__)


In [ ]:
covid19_genome = Covid19Genome()

lineages = covid19_genome.getLocalLineages(0)
lineages.sort()
dataset = []
for lineage in lineages[:10]:
    for accession in covid19_genome.getLocalAccessionsPath(lineage):
        dataset.append((accession, lineage))

In [1]:
from prod_reads_ds import ProdReadsDS
from mh_reads_ds import Dataset as MHReadsDS
import numpy as np

2023-08-19 16:04:01.131165: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-19 16:04:01.156104: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 16:04:01.602613: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
prod_reads_ds = ProdReadsDS(kmer_len=16, frag_len=86, num_frags=400)

2023-08-19 16:04:02.090959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 16:04:02.106263: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 16:04:02.106377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
reads_ds = prod_reads_ds.get_tf_dataset(dirpath='data/reads/')
genome_ds = MHReadsDS(mapping=[("data/genomes/OV876520.fasta", "1")]).get_tf_dataset()

2023-08-19 16:04:02.673855: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_0' with dtype string
	 [[{{node args_0}}]]
2023-08-19 16:04:02.815324: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_0' with dtype string
	 [[{{node args_0}}]]
2023-08-19 16:04:02.815367: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_1' with dtype bool and shape [?]
	 [[{{node args_1}}]]


In [4]:
for elem, label in reads_ds.take(1):
    read_obj = elem[0].numpy()
    print(read_obj)

2023-08-19 16:04:02.850484: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-19 16:04:02.850581: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


[b'...']


In [5]:
for elem,label in genome_ds.take(1):
    genome_obj = elem[0].numpy()
    print(genome_obj)

2023-08-19 16:04:03.990464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype string and shape [1]
	 [[{{node Placeholder/_12}}]]
2023-08-19 16:04:03.990713: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype resource
	 [[{{node Placeholder/_8}}]]


[b'...']


In [ ]:
for i in range(read_obj.shape[0]):
    res = genome_obj * read_obj[i]
    # print(f" the shape of the result is: {res.shape}")
    for j in range(res.shape[0]):
        similarity = (np.sum(res[j]) / 86)
        # print(similarity)
        if similarity > 0.5:
            print(f"the {j}th fragment from the genome is similar to the {i}th fragment from the read")


In [ ]:
import tensorflow as tf
print(tf.random.stateless_uniform([1], minval=0, maxval=4**16, dtype=tf.int64, seed=[0,0]))
print(tf.random.stateless_uniform([1], minval=0, maxval=4**16, dtype=tf.int64, seed=[0,0]))
print(tf.random.stateless_uniform([1], minval=0, maxval=4**16, dtype=tf.int64, seed=[0,0]))

In [8]:
######################################
#### test signature compatibility ####
######################################
import tensorflow as tf

In [9]:
kmer = "AACGATACGCTAGCTG"
kmer_tensor = tf.strings.bytes_split([kmer])

In [22]:
# define hash table
keys = tf.constant(["A", "C", "G", "T"])
values = tf.constant([1, 2, 3, 4], dtype=tf.int32)
default_value = tf.constant(0, dtype=tf.int32)
initializer = tf.lookup.KeyValueTensorInitializer(keys, values)

# Define the hash table
table = tf.lookup.StaticHashTable(initializer, default_value)

# Look up the values
mapped_genome = table.lookup(tf.strings.bytes_split(kmer))
kmers_signature = tf.zeros(shape=(1), dtype=tf.int64)

for i in range(16):
    print(tf.cast(tf.roll(mapped_genome, shift=-1*i, axis=0)[:1], tf.int64))
    kmers_signature += tf.cast(tf.roll(mapped_genome, shift=-1*i, axis=0)[:1] * tf.pow(5,i), tf.int64)

tf.Tensor([1], shape=(1,), dtype=int64)
tf.Tensor([1], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([3], shape=(1,), dtype=int64)
tf.Tensor([1], shape=(1,), dtype=int64)
tf.Tensor([4], shape=(1,), dtype=int64)
tf.Tensor([1], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([3], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([4], shape=(1,), dtype=int64)
tf.Tensor([1], shape=(1,), dtype=int64)
tf.Tensor([3], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([4], shape=(1,), dtype=int64)
tf.Tensor([3], shape=(1,), dtype=int64)


In [15]:
kmers_signature

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([-1025305107])>

In [23]:
base_to_int =  {'A': 1, 'C': 2, 'G': 3, 'T': 4}

kmer_signature2 = 0
for i in range(16):
    print(base_to_int.get(kmer[i], 0))
    kmer_signature2 += base_to_int.get(kmer[i], 0) * (5**i)

1
1
2
3
1
4
1
2
3
2
4
1
3
2
4
3


In [18]:
print(kmer_signature2)

119233779181
